# oneTBB task_scheduler_observer

##### Sections
- [An overview of task_scheduler_obsever](#The-partitioners-available-in-oneTBB)
- _Code_: [Observe when threads enter an explicit task_arena](#Observe-when-threads-enter-an-explicit-task_arena)

## Learning Objectives
* Learn about how to create and use a task_scheduler_observer in oneTBB

## An overview of task_scheduler_observer

In oneTBB, we can derive an observer from `task_scheduler_observer` and define
functions that are called when threads enter and leave the oneTBB scheduler, or
enter and leave a specific task arena.

We can use these functions to trace program execution or to call OS-specific functions
for pinning and affinitization.

In this module, we will use a `task_scheduler_observer` to record what `task_arena` slot
a thread is associated with when it enters an explicit task arena. We can query the
slot number with the function `tbb::this_task_arena::current_thread_index()`. In a real
application, we might use this information to pin the thread to a specific core or set
of cores.

## Observe when threads enter an explicit task_arena

In this exercise, we execute a `tbb::parallel_for` that has a check in
each iteration. The iteration checks if the thread local variable my_tid 
has been set and matches the result of `tbb::this_task_arena::current_thread_index()`. 
In the implementation that is provided, this check will fail, ending the program
early.

The exercise is to fix the `tid_observer` class and use it to properly set
the thread local value of `my_tid` for each thread that participates in 
executing iterations of the `parallel_for`.

The interface of `task_scheduler_observer` is provided below.

```cpp
// Defined in header <tbb/task_scheduler_observer.h>

namespace tbb {

   class task_scheduler_observer {
   public:
       task_scheduler_observer();
       explicit task_scheduler_observer( task_arena& a );
       virtual ~task_scheduler_observer();

       void observe( bool state=true );
       bool is_observing() const;

       virtual void on_scheduler_entry( bool is_worker ) {}
       virtual void on_scheduler_exit( bool is_worker } {}
   };

}
```

Perform the following steps to complete this exercise:
1. Inspect the code cell below, then click run ▶ to save the code to a file
2. Run ▶ the cell in the __Build and Run__ section below the code snippet to compile and execute the code. The loop should not complete since my_tid will not be set correctly.
3. Modify the code so that it correctly associates a thread index with each thread that enters the task_arena.
  1. In the `tid_observer` class, fix the `INCORRECT_VALUE`.
  2. In the `main` function, construct a `tid_observer` object to observe `task_arena a`.
4. Run ▶ the cell in the __Build and Run__ section again compile and execute the modified code. The loop should now complete. 

In [ ]:
%%writefile lab/observer.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <exception>
#include <iostream>
#include <tbb/tbb.h>

#define INCORRECT_VALUE -1
thread_local int my_tid = INCORRECT_VALUE;

class tid_observer : public tbb::task_scheduler_observer {
public:
  tid_observer(tbb::task_arena &a) : tbb::task_scheduler_observer{a} { observe(true); }
  void on_scheduler_entry( bool is_worker ) override {
    // STEP 3.A: replace the INCORRECT_VALUE with a call to tbb::this_task_arena::current_thread_index()
    my_tid = INCORRECT_VALUE;
  }
};

int main() {
  tbb::task_arena a;
  // STEP 3.B: construct a tid_observer, passing "a" to the constructor
  try {
    a.execute([]() {
      tbb::parallel_for(0, 1000000, [](int i) {
        const int time_per_iteration = 1000; // 1 us
        auto t0 = std::chrono::high_resolution_clock::now();
        while ((std::chrono::high_resolution_clock::now() - t0).count() < time_per_iteration);
        if (my_tid == -1 || my_tid != tbb::this_task_arena::current_thread_index()) {
          std::cout << "my_tid not set correctly on entry!\n";
          throw std::exception{};
        }
      });
    });
  } catch (...) {
    std::cout << "Did not complete loop.\n";
    return 1;
  }
  std::cout << "Completed loop.\n";
  return 0;
}

In [ ]:
!chmod 755 q; chmod 755 ./scripts/run_observer.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_observer.sh; else ./scripts/run_observer.sh; fi

## Solution (Don't peak unless you have to)

In [ ]:
%%writefile solutions/observer-solved.cpp
//==============================================================
// Copyright (c) 2020 Intel Corporation
//
// SPDX-License-Identifier: Apache-2.0
// =============================================================

#include <exception>
#include <iostream>
#include <tbb/tbb.h>

#define INCORRECT_VALUE -1
thread_local int my_tid = INCORRECT_VALUE;

class tid_observer : public tbb::task_scheduler_observer {
public:
  tid_observer(tbb::task_arena &a) : tbb::task_scheduler_observer{a} { observe(true); }
  void on_scheduler_entry( bool is_worker ) override {
    // STEP 3.A: replace the INCORRECT_VALUE with a call to tbb::this_task_arena::current_thread_index()
    my_tid = tbb::this_task_arena::current_thread_index();
  }
};

int main() {
  tbb::task_arena a;
  // STEP 3.B: construct a tid_observer, passing "a" to the constructor
  tid_observer to(a);
  try {
    a.execute([]() {
      tbb::parallel_for(0, 1000000, [](int i) {
        const int time_per_iteration = 1000; // 1 us
        auto t0 = std::chrono::high_resolution_clock::now();
        while ((std::chrono::high_resolution_clock::now() - t0).count() < time_per_iteration);
        if (my_tid == -1 || my_tid != tbb::this_task_arena::current_thread_index()) {
          std::cout << "my_tid not set correctly on entry!\n";
          throw std::exception{};
        }
      });
    });
  } catch (...) {
    std::cout << "Did not complete loop.\n";
    return 1;
  }
  std::cout << "Completed loop.\n";
  return 0;
}

In [ ]:
!chmod 755 q; chmod 755 ./scripts/run_observer-solved.sh; if [ -x "$(command -v qsub)" ]; then ./q scripts/run_observer-solved.sh; else ./scripts/run_observer-solved.sh; fi